# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCE


In [3]:
# Initialize

openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [6]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

## And then enter Gradio's magic!

In [7]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'hi'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'content': 'joke please?'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'joke please?', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Sure! Here’s one for you:\n\nWhy did the scarecrow win a

In [8]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [32]:
def simple_chat(message):
    messages = [{"role": "system", "content": system_message}] + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [33]:
print(gr.__version__)

5.22.0


In [36]:
import gradio as gr
from typing import List, Dict, Any, Tuple, Optional

# -------------------------
# Build a Gemini/Perplexity-like shell around your chat(messages) fn
# -------------------------
def build_chat_ui(chat_fn):
    CSS = """
    body, .gradio-container { background: radial-gradient(1200px 600px at 20% -10%, #c9e7ff55, transparent 60%),
                                          radial-gradient(1200px 600px at 90% 10%, #ffd6e955, transparent 60%),
                                          linear-gradient(180deg, #0f1115, #0b0d12); }
    .topbar { backdrop-filter: blur(10px); background: rgba(255,255,255,0.06); border: 1px solid rgba(255,255,255,0.08);
              box-shadow: 0 10px 30px rgba(0,0,0,0.2); border-radius: 20px; padding: 14px 18px; }
    .brand { display: flex; align-items: center; gap: 10px; color: #e8eefb; font-weight: 600; }
    .brand .dot { width: 10px; height: 10px; border-radius: 999px; background: #8ab4ff; box-shadow: 0 0 12px #8ab4ff; }
    .subtitle { color: #9aa5b1; font-size: 0.92rem; }
    .wrap { max-width: 1100px; margin: 0 auto; }
    .chatpanel, .sidepanel { backdrop-filter: blur(8px); background: rgba(255,255,255,0.04);
                              border: 1px solid rgba(255,255,255,0.08); box-shadow: 0 10px 30px rgba(0,0,0,0.25); border-radius: 16px; }
    .gr-chat-message { max-width: 780px; margin-left: auto; margin-right: auto; }
    .gr-chat-message.user { background: rgba(138, 180, 255, 0.10) !important; border: 1px solid rgba(138, 180, 255, 0.25); }
    .gr-chat-message.bot  { background: rgba(255, 255, 255, 0.04) !important; border: 1px solid rgba(255, 255, 255, 0.10); }
    .gr-chat-message p, .gr-chat-message li { color: #e9edf6; line-height: 1.65; }
    .gr-chat-message code { background: #0b1322; border: 1px solid #14223c; padding: 2px 6px; border-radius: 6px; }
    .gr-chat-message pre code { display: block; padding: 12px; border-radius: 10px; }
    .gr-textbox textarea { min-height: 56px; font-size: 16px; line-height: 1.35; }
    .gr-button { border-radius: 999px; }
    .prompt-chip button { width: 100%; text-align: left; background: rgba(255,255,255,0.03);
                          border: 1px solid rgba(255,255,255,0.08); border-radius: 12px; }
    .prompt-chip button:hover { background: rgba(255,255,255,0.06); }
    .footnote { color: #8a94a6; font-size: 12px; text-align: center; padding: 6px 0 2px; }
    """

    suggested_prompts = [
        "Summarize this article in 3 bullets (paste URL):",
        "Explain like I'm 5: what is vector DB vs RDBMS?",
        "Draft a polite follow-up email for an interview",
        "Why did my SQL query get slower after adding an index?"
    ]

    # ---- Helpers to keep messages in OpenAI-style schema (type="messages")
    def to_msg(role: str, content: str) -> Dict[str, Any]:
        return {"role": role, "content": content}

    # Submit: append user -> call chat_fn(messages) -> append assistant
    def on_submit(user_text: str,
                  messages: List[Dict[str, Any]],
                  smart: bool,
                  concise: bool):
        user_text = (user_text or "").strip()
        if not user_text:
            return gr.update(), messages  # no change
        new_msgs = messages + [to_msg("user", user_text)]

        # If you want the toggles to reach your model, you can inject system hints here:
        sys_hints = []
        if smart:
            sys_hints.append("SmartSearch=on")
        if concise:
            sys_hints.append("Concise=on")
        if sys_hints:
            new_msgs = [to_msg("system", " ; ".join(sys_hints))] + new_msgs

        reply = chat_fn(new_msgs)  # your function
        # Normalize reply to string
        if isinstance(reply, list):
            # If your fn returns messages, take the last assistant string
            last_assistant = next((m for m in reversed(reply) if m.get("role") == "assistant"), None)
            content = last_assistant.get("content") if last_assistant else ""
        else:
            content = str(reply)

        new_msgs = messages + [to_msg("user", user_text), to_msg("assistant", content)]
        return "", new_msgs  # clear textbox, update messages

    # Retry: remove last assistant and re-call with same last user
    def on_retry(messages: List[Dict[str, Any]]):
        if not messages:
            return messages
        # Remove trailing assistant if present
        msgs = messages[:]
        if msgs[-1]["role"] == "assistant":
            msgs.pop()
        # Find the last contiguous (user ... ) segment to re-ask
        # If the last is user, just call once; if assistant removed, last should be user.
        if not msgs or msgs[-1]["role"] != "user":
            return messages  # nothing to retry
        reply = chat_fn(msgs)
        if isinstance(reply, list):
            last_assistant = next((m for m in reversed(reply) if m.get("role") == "assistant"), None)
            content = last_assistant.get("content") if last_assistant else ""
        else:
            content = str(reply)
        msgs.append(to_msg("assistant", content))
        return msgs

    # Undo: pop last message (like Perplexity’s Undo)
    def on_undo(messages: List[Dict[str, Any]]):
        if not messages:
            return messages
        return messages[:-1]

    # Clear
    def on_clear():
        return []

    with gr.Blocks(css=CSS, fill_height=True, theme=None) as demo:
        with gr.Column(elem_classes=["wrap"]):
            with gr.Row(elem_classes=["topbar"], equal_height=True):
                with gr.Column(scale=6):
                    gr.HTML(
                        """
                        <div class="brand">
                            <div class="dot"></div>
                            <div>Nova Chat</div>
                        </div>
                        <div class="subtitle">A crisp, focused chat workspace — ask anything.</div>
                        """
                    )
                with gr.Column(scale=6, min_width=200):
                    web_toggle = gr.Checkbox(value=True, label="Smart Search")
                    concise_toggle = gr.Checkbox(value=False, label="Concise Mode")

            with gr.Row():
                with gr.Column(scale=5, elem_classes=["chatpanel"]):
                    messages_state = gr.State([])  # list[{"role":..., "content":...}]
                    chatbox = gr.Chatbot(
                        height=560,
                        type="messages",
                        show_copy_button=True,
                        label=None,
                        value=[]
                    )
                    textbox = gr.Textbox(
                        placeholder="Ask anything…  ⏎ to send   |  Shift+⏎ for new line",
                        autofocus=True,
                        lines=3,
                        submit_btn=gr.Button("Send", variant="primary"),
                        show_label=False
                    )
                    with gr.Row():
                        retry_btn = gr.Button("↻ Retry", size="sm")
                        undo_btn  = gr.Button("↩ Undo", size="sm")
                        clear_btn = gr.Button("🧹 Clear", size="sm")

                with gr.Column(scale=2, min_width=260, elem_classes=["sidepanel"]):
                    gr.Markdown("### Quick prompts")
                    for p in suggested_prompts:
                        gr.Button(f"• {p}", elem_classes=["prompt-chip"]).click(
                            lambda t=p: t, outputs=textbox
                        )
                    gr.Markdown("---\n### Tips\n- Toggle **Smart Search** for web-aware replies.\n- Turn on **Concise Mode** for TL;DR answers.")
                    gr.HTML('<div class="footnote">UI shell inspired by Gemini & Perplexity · Built with Gradio</div>')

        # --- Wiring
        # Submit flow: textbox -> on_submit -> (textbox empty, messages updated) -> reflect to Chatbot
        textbox.submit(
            on_submit,
            inputs=[textbox, messages_state, web_toggle, concise_toggle],
            outputs=[textbox, messages_state],
        ).then(
            lambda msgs: msgs,  # mirror state to Chatbot
            inputs=[messages_state],
            outputs=[chatbox],
        )

        # Retry
        retry_btn.click(
            on_retry, inputs=[messages_state], outputs=[messages_state]
        ).then(
            lambda msgs: msgs, inputs=[messages_state], outputs=[chatbox]
        )

        # Undo
        undo_btn.click(
            on_undo, inputs=[messages_state], outputs=[messages_state]
        ).then(
            lambda msgs: msgs, inputs=[messages_state], outputs=[chatbox]
        )

        # Clear
        clear_btn.click(
            on_clear, outputs=[messages_state]
        ).then(
            lambda msgs: msgs, inputs=[messages_state], outputs=[chatbox]
        )

    return demo


# -------------------------
# Example usage
# -------------------------
# def chat(messages):
#     # messages: list of {"role": "...", "content": "..."}
#     return "Hello from your model!"
#
# demo = build_chat_ui(chat)
# demo.launch()


In [37]:
demo=build_chat_ui(simple_chat)
demo.launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


In [10]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [11]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


In [14]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>